In [ ]:
# openssl req -x509 -newkey rsa:4096 -nodes -keyout key.pem -out cert.pem -days 365

In [1]:
import csv
from datetime import datetime
import pandas as pd
import random
from datetime import datetime, timedelta
import pytz

In [2]:
df = pd.read_csv('../downloads/synthetic_gets.csv')

def random_date_in_last_30_days():
    today = datetime.now()
    random_days_ago = today - timedelta(days=random.randint(0, 30))
    random_time = random_days_ago.replace(hour=random.randint(0, 23),
                                          minute=random.randint(0, 59),
                                          second=random.randint(0, 59))
    return random_time

df['mincolldate'] = df['mincolldate'].apply(lambda x: random_date_in_last_30_days())

df['mincolldate'] = df['mincolldate'].apply(lambda x: x if x.tzinfo else x.replace(tzinfo=pytz.UTC))
df['mincolldate'] = df['mincolldate'].apply(lambda x: x.isoformat().replace('+00:00', 'Z') if x.tzinfo == pytz.UTC else x.isoformat())

df.to_csv('../downloads/synthetic_gets.csv', index=False)

In [3]:
csv_file = '../downloads/synthetic_gets.csv'
json_data = []
with open(csv_file, mode='r', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        json_data.append(row)

In [4]:
from flask import Flask, request, jsonify
app = Flask(__name__)

In [ ]:
def filter_data(data, filter_criteria):
    for key, value in filter_criteria.items():
        if '..' in value:
            min_val, max_val = value.split('..')
            if 'T' in min_val:  # Handling datetime range
                min_val = datetime.fromisoformat(min_val.replace('Z', '+00:00'))
                max_val = datetime.fromisoformat(max_val.replace('Z', '+00:00'))
                data = [item for item in data if key in item and min_val <= datetime.fromisoformat(item[key].replace('Z', '+00:00')) <= max_val]
            else:  # Handling numeric range
                min_val, max_val = float(min_val), float(max_val)
                data = [item for item in data if key in item and min_val <= float(item[key]) <= max_val]
        else:
            # Handle exact match filters
            data = [item for item in data if key in item and item[key] == value]
    return data

@app.route('/filter', methods=['GET'])
def filter_json():
    try:
        filter_criteria = request.args.to_dict()
        filtered_data = filter_data(json_data, filter_criteria)
        return jsonify(filtered_data)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, ssl_context=('cert.pem', 'key.pem'))

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:5000
 * Running on https://172.20.3.113:5000
Press CTRL+C to quit
172.20.8.42 - - [03/Feb/2024 22:42:39] "GET /filter?mincolldate=2023-01-01T00:00:00.000Z..2023-12-31T23:59:59.000Z&declat=22.05316595745477..27.836047242545227&declon=118.21456665930681..124.59238234069319 HTTP/1.1" 200 -
172.20.8.42 - - [03/Feb/2024 22:44:10] "GET /filter?mincolldate=2023-01-01T00:00:00.000Z..2023-12-31T23:59:59.000Z&declat=20.607470636182157..29.28174256381784&declon=116.62013773896022..126.18681126103978 HTTP/1.1" 200 -
172.20.8.42 - - [03/Feb/2024 22:49:52] "GET /filter?mincolldate=2023-01-01T00:00:00.000Z..2023-12-31T23:59:59.000Z&declat=20.607470636182157..29.28174256381784&declon=116.62013773896022..126.18681126103978 HTTP/1.1" 200 -
172.20.8.42 - - [03/Feb/2024 22:52:29] "GET /filter?mincolldate=2024-01-01T00:00:00.000Z..2024-02-03T22:52:27.000Z&declat=20.607470636182157..29.28174256381784&declon=116.62013773896022..126.1868112